In [231]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.dates as mpl_dates
from xgboost import XGBRFRegressor
from sklearn.preprocessing import OrdinalEncoder
import warnings
warnings.simplefilter(action='ignore', category=Warning)



In [232]:
#develop a machine learning model that helps forcast the cases in the future 

In [233]:
#datas 

In [234]:
#github data source 

ECDC = pd.read_csv("/Users/anuragshrestha/Desktop/covid-19-data/public/data/ecdc/COVID-2019 - ECDC (2020).csv")
full_data = pd.read_csv('/Users/anuragshrestha/Desktop/covid-19-data/public/data/ecdc/full_data.csv') 
df_locations = pd.read_csv('/Users/anuragshrestha/Desktop/covid-19-data/public/data/ecdc/locations.csv')
NewCases = pd.read_csv('/Users/anuragshrestha/Desktop/covid-19-data/public/data/ecdc/new_cases.csv')
Deaths = pd.read_csv("/Users/anuragshrestha/Desktop/covid-19-data/public/data/ecdc/total_deaths.csv")

ecdc_lastest = pd.read_csv('/Users/anuragshrestha/Desktop/covid-19-data/input/ecdc/releases/latest.csv')




#kaggle Covid19 forecasting challenge data sets 

train_set = pd.read_csv('/Users/anuragshrestha/Desktop/countryinfo/covid19-global-forecasting-week-3/train.csv')
test_set = pd.read_csv('/Users/anuragshrestha/Desktop/countryinfo/covid19-global-forecasting-week-3/test.csv')
submission = pd.read_csv('/Users/anuragshrestha/Desktop/countryinfo/covid19-global-forecasting-week-3/submission.csv')

#datasets from the community 

CountryInfo = pd.read_csv('/Users/anuragshrestha/Desktop/countryinfo/covid19countryinfo.csv')
CountryTest = pd.read_csv('/Users/anuragshrestha/Desktop/countryinfo/covid19tests.csv')
LocationInfo = pd.read_csv('/Users/anuragshrestha/Desktop/countryinfo/locations_population.csv')
Recovered = pd.read_csv('/Users/anuragshrestha/Desktop/countryinfo/novel-corona-virus-2019-dataset/covid_19_data.csv')



                                     

             Data Pre-processing 

In [235]:
 
train_set['Date'] = pd.to_datetime(train_set['Date'], format = '%Y-%m-%d')
test_set['Date'] = pd.to_datetime(test_set['Date'], format = '%Y-%m-%d')


def dayMonth(df):
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day 
    return df


#encoding the country and states into numerical values 
#replacing the missing province state with just country name for now 

def encoder(df): 
    df.Province_State.fillna(value = 'NaN', inplace = True)
    label_maker = OrdinalEncoder()
    df[['Province_State','Country_Region']] = label_maker.fit_transform(df.loc[:,['Province_State','Country_Region']])
    #df.loc[:, 'Date'] = df['Date'].dt.strftime("%m%d")
    #df['Date'] = df['Date'].astype('int')
    return df 

for df in [train_set, test_set]:
    dayMonth(df)
    encoder(df)


In [236]:
#training series before the date of 03.26
train_set = train_set[train_set['Date'] < '2020-03-26']
    
#features for now 

features_train = ['Province_State', 'Country_Region', 'Month', 'Day', 'ConfirmedCases', 'Fatalities']

df_train = train_set[features_train]

features_test = ['Province_State', 'Country_Region', 'Month', 'Day']



Timestamp('2020-03-26 00:00:00')

In [237]:
#Model building 

#get unique values for the coutries 

prediction = []

for country in df_train.Country_Region.unique(): 
    df_Train = df_train[df_train['Country_Region'] == country]
    
    for state in df_train.Province_State.unique(): 
        df_Train = df_train[df_train['Province_State'] == state]
        Train = df_Train.values
        xtrain, ytrain = Train[:, :-2], Train[:,-2:]
        
        XGB_1 = XGBRFRegressor(n_estimators= 1100) #for confirmed cases
        XGB_1.fit(xtrain, ytrain[:,0])
        
        XGB_2 = XGBRFRegressor(n_estimators= 1100)
        XGB_2.fit(xtrain, ytrain[:,1])     #for fatalities 
        
        xtest1 = test_set[(test_set['Country_Region'] == country) & (test_set['Province_State'] == state)]
        ForeCastID = xtest1.ForecastId.values 
        
        xtest2 = xtest1[features_test]
        
        #predictions of 2 output in the matrix
        cases_pred = np.round(XGB_1.predict(xtest2.values),5)
        death_pred = np.round(XGB_2.predict(xtest2.values),5)
        
        #Append the predicted values to submission list
        for i in range(len(cases_pred)):
            d = {'ForecastId': ForeCastID[i], 'ConfirmedCases': cases_pred[i], 'Fatalities':death_pred[i]}
            prediction.append(d)
    

[21:45:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:45:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: 

In [ ]:
submission = pd.DataFrame(prediction)

submission.to_csv('BasePrediction.csv', index = False)